In [38]:
import requests
from requests.auth import HTTPBasicAuth
import json
from secret_data import userpass 
import pandas as pd 
import re 
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
import json
from datetime import datetime

# Load CSV and normalize column names
odp = pd.read_csv('ersi_final.csv')
odp.columns = odp.columns.str.strip().str.lower()

# Credentials
domain = "opendata.maryland.gov"
username = userpass[0]
password = userpass[1]

# Today's date in YYYY-MM-DD format
today_date = datetime.today().strftime('%Y-%m-%d')

for index, row in odp.iterrows():
    dataset_id = row['uid']
    metadata_url = f"https://{domain}/api/views/{dataset_id}"
    
    # Retrieve current metadata
    response = requests.get(metadata_url, auth=HTTPBasicAuth(username, password))
    
    if response.status_code != 200:
        print(f"Failed to retrieve metadata: {response.status_code}, {response.text}, {dataset_id}")
        continue

    metadata = response.json()
    
    # Get current custom_fields safely
    custom_fields = metadata.get("metadata", {}).get("custom_fields", {})

    # Preserve existing sections
    jurisdiction_section = custom_fields.get("Jurisdiction", {})
    time_period_section = custom_fields.get("Time Period", {})
    agency_section = custom_fields.get("Agency", {})
    gis_section = custom_fields.get("GIS Download", {})

    # Update fields
    jurisdiction_section["Jurisdiction"] = "State of Maryland"
    time_period_section["Update Frequency"] = "As Needed"
    time_period_section["Date Metadata Written"] = today_date
    time_period_section["Time Period of Content"] = row['create_date']
    agency_section["State Agency Performing Data Updates"] = "Department of Information Technology"
    gis_section["Place Keywords"] = "Maryland"
        
    # Rebuild custom_fields
    custom_fields["Jurisdiction"] = jurisdiction_section
    custom_fields["Time Period"] = time_period_section
    custom_fields["GIS Download"] = gis_section
    custom_fields["Agency"]= agency_section

    # Step 2: Append the attributionLink sentence to description if not already present
    description_text = metadata.get("description", "")
    attribution_sentence = f" Feature link: {metadata.get('attributionLink', '')}"
    if attribution_sentence.strip() not in description_text:
        description_text = description_text.rstrip() + attribution_sentence

    # Build payload
    payload = {
        "description": description_text,
        "attributionLink": row['ersi_link'],
        "metadata": {
            "custom_fields": custom_fields
        }
    }

    # Send update
    update_response = requests.put(
        metadata_url,
        auth=HTTPBasicAuth(username, password),
        headers={"Content-Type": "application/json"},
        data=json.dumps(payload)
    )

    if update_response.status_code in [200, 202]:
        print(f"{dataset_id} → Updated: Jurisdiction, Update Frequency, Date Metadata Written = {today_date}")
    else:
        print(f"Failed to update: {update_response.status_code}, {update_response.text}, {dataset_id}")






nw2g-v2vv → Updated: Jurisdiction, Update Frequency, Date Metadata Written = 2025-08-11
x5si-a5i6 → Updated: Jurisdiction, Update Frequency, Date Metadata Written = 2025-08-11
qzca-qdke → Updated: Jurisdiction, Update Frequency, Date Metadata Written = 2025-08-11
qpxv-23p9 → Updated: Jurisdiction, Update Frequency, Date Metadata Written = 2025-08-11
w4cj-zxrk → Updated: Jurisdiction, Update Frequency, Date Metadata Written = 2025-08-11
hpun-upnm → Updated: Jurisdiction, Update Frequency, Date Metadata Written = 2025-08-11
a9hq-vadx → Updated: Jurisdiction, Update Frequency, Date Metadata Written = 2025-08-11
rcge-ybut → Updated: Jurisdiction, Update Frequency, Date Metadata Written = 2025-08-11
r5my-n834 → Updated: Jurisdiction, Update Frequency, Date Metadata Written = 2025-08-11
sgap-qdpw → Updated: Jurisdiction, Update Frequency, Date Metadata Written = 2025-08-11
qz6v-cy3q → Updated: Jurisdiction, Update Frequency, Date Metadata Written = 2025-08-11
bxrt-et9r → Updated: Jurisdictio